# Model Fine Tuning
----
**Objective**: In this notebook, you will get to fine tune a transformer model on MRPC dataset. You will get to go through the whole process of transfer learning from loading the dataset, loading the model, compiling the model, training the model, and finally evaluating your results.

NOTE: Make sure to change the runtime from CPU to TPU or GPU for faster training

## Install Libraries
Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## Import Dataset
___
In this section we will use as an example the MRPC (Microsoft Research Paraphrase Corpus) dataset. The dataset consists of 5,801 pairs of sentences, with a label indicating if they are paraphrases or not (i.e., if both sentences mean the same thing).
This is one of the 10 datasets composing the GLUE benchmark, which is an academic benchmark that is used to measure the performance of ML models across 10 different text classification tasks.


**Question 1**: Use the "load_datasets" function to load the "mrpc" dataset from the "glue" benchmark.

In [23]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc") # Add your solution here

## Pre-processing data

First, we need to tokenize our data. We will do so using the BERT tokenizer. We will use the AutoTokenizer with truncation.
This will truncate token by token, removing a token from the longest sequence in the pair if a pair of sequences (or a batch of pairs) is provided.

**Question 2:** Use the .map() function to apply the tokenization functionon the raw dataset. Hint: set the batched flag to True for faster processing.

In [24]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) # Add your solution here

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Then, we need to pad our data so that all input sequences have the same lengths.

In [25]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

Lets take a training example to see what our dataset looks like.

**Question 3:** Display an example from the training section of the dataset. Hint: The tokenized_datasets variable is a nested dictionary where the main keys are "train", "test", and "validate"

In [26]:
tokenized_datasets["train"][10]  # Add your solution here

{'sentence1': 'Legislation making it harder for consumers to erase their debts in bankruptcy court won overwhelming House approval in March .',
 'sentence2': 'Legislation making it harder for consumers to erase their debts in bankruptcy court won speedy , House approval in March and was endorsed by the White House .',
 'label': 0,
 'idx': 11,
 'input_ids': [101,
  6094,
  2437,
  2009,
  6211,
  2005,
  10390,
  2000,
  22505,
  2037,
  13930,
  1999,
  10528,
  2457,
  2180,
  10827,
  2160,
  6226,
  1999,
  2233,
  1012,
  102,
  6094,
  2437,
  2009,
  6211,
  2005,
  10390,
  2000,
  22505,
  2037,
  13930,
  1999,
  10528,
  2457,
  2180,
  26203,
  1010,
  2160,
  6226,
  1999,
  2233,
  1998,
  2001,
  11763,
  2011,
  1996,
  2317,
  2160,
  1012,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

Finally, we will prepare our training and validation datasets by transforming them into tensorflow datasets.

In [27]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Model Preperation
---
In this section, we will load the model and equip it with a classifier head. This model is already pre-trained and we will fine tune it with the dataset for paraphrasing classification that we prepared.

### Load Classifier Head

In [28]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Compile the model

We will compile the model using SparseCategoricalCrossentropyloss, which is when there are two or more label classes that are not one hot encded.

**Question 4:** Compile the model using the adam optimizer, SparseCategoricalCrossentropy loss (with from_logits flag set to True) and displaying the accuracy metric.

In [29]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Add your solution here
model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

### Train the model

Now we will train our model using the processing training and validation sets.

**Question 5:** Fit the model on the training and validation sections that we prepared earlier.

In [30]:
# Add your solution here
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=5
)

Epoch 1/5
459/459 [==============================] - 149s 201ms/step - loss: 0.6627 - accuracy: 0.6499 - val_loss: 0.6407 - val_accuracy: 0.6838
Epoch 2/5
459/459 [==============================] - 77s 167ms/step - loss: 0.6428 - accuracy: 0.6704 - val_loss: 0.6436 - val_accuracy: 0.6838
Epoch 3/5
459/459 [==============================] - 74s 161ms/step - loss: 0.6390 - accuracy: 0.6748 - val_loss: 0.6272 - val_accuracy: 0.6838
Epoch 4/5
459/459 [==============================] - 74s 161ms/step - loss: 0.6361 - accuracy: 0.6723 - val_loss: 0.6256 - val_accuracy: 0.6838
Epoch 5/5
459/459 [==============================] - 74s 161ms/step - loss: 0.6419 - accuracy: 0.6701 - val_loss: 0.6254 - val_accuracy: 0.6838


## Evaluating the model

Let's test how accurate the models are! In this section, we will predict on the testing dataset to check the model's performance when it comes to classifying paraphrased sentences on unseen data.

In [31]:
preds = model.predict(tf_test_dataset)["logits"]

216/216 [==============================] - 14s 54ms/step


In [32]:
import numpy as np
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(1725, 2) (1725,)


In [33]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=class_preds, references=raw_datasets["test"]["label"])

{'accuracy': 0.664927536231884, 'f1': 0.7987465181058496}